## Restaurant Evaluator

Shuyang specify data
Tokenize
Split data (80 training, 10 test, 10 validation)


### neural network

### train the model

Cornelius output a vector of probabilities (we only care about the probability that it's positive)
Validate.
